# Installing and importing required dependencies

In [1]:
!pip install -q llama-index matplotlib
!pip install -q llama-index-llms-groq
!pip install -q llama-index-embeddings-huggingface

In [12]:
# Import nest_asyncio to allow nested event loops
import nest_asyncio
nest_asyncio.apply()

# Access the secret in the SingleStore notebook
from singlestoredb.management import get_secret

# Import necessary libraries and modules
from llama_index.llms.groq import Groq  # Import Groq from llama_index for Text-to-SQL processing
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding # Import Hugging Face embedding model
from sqlalchemy import *  # Import SQLAlchemy for database interaction
from llama_index.core import SQLDatabase  # Import SQLDatabase for handling database operations
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine  # Import engine for NL to SQL query processing
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,  # Import query engine for SQL table retrieval
)
from llama_index.core.objects import (
    SQLTableNodeMapping,  # Import mapping for SQL table nodes
    ObjectIndex,  # Import ObjectIndex for indexing database objects
    SQLTableSchema,  # Import schema definition for SQL tables
)
from llama_index.core import VectorStoreIndex  # Import VectorStoreIndex for indexing vectors
from llama_index.core.retrievers import NLSQLRetriever  # Import retriever for NL to SQL conversion
from llama_index.core.response.notebook_utils import display_source_node  # Import utility to display source nodes in notebooks
from llama_index.core.query_engine import RetrieverQueryEngine  # Import general query engine for data retrieval

In [13]:
#Getting the secret 

secret = get_secret('groq')


In [14]:
#Set the LLM model to Meta Llama 3.2
llm = Groq(model="llama-3.2-3b-preview", api_key = secret)


In [15]:
#set the embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = llm
Settings.embed_model = embed_model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
#Let's connect the engine 

engine = create_engine("singlestoredb://"+"admin"+":"+"ehlhXaU0v7yBifZt96lu1g25UDRMmMJZ"+"@"+"svc-bc090a30-95a8-4de0-bd17-3887390b2d1a-dml.azr-pune-1.svc.singlestore.com"+":"+"3306"+"/"+"Stocks")
sql_database = SQLDatabase(engine)

In [19]:
# Initialize the NLSQLTableQueryEngine with the specified SQL database, tables, and language model
query_engine = NLSQLTableQueryEngine(
 sql_database=sql_database, # The SQL database instance to query
 tables=["stock_tweets", "stock_yfinance_data"], # List of tables to include in the query engine
 llm=llm, # The language model used for processing natural language queries
)


In [20]:
# Initialize the SQLTableNodeMapping with the sql_database
table_node_mapping = SQLTableNodeMapping(sql_database)
# Define table schemas
table_schema_objs = [
 SQLTableSchema(table_name="stock_tweets"),
 SQLTableSchema(table_name="stock_yfinance_data")
]
# Create the ObjectIndex from the schema objects and node mapping
obj_index = ObjectIndex.from_objects(
 table_schema_objs,
 table_node_mapping,
 VectorStoreIndex,
)
# Initialize the SQLTableRetrieverQueryEngine
query_engine = SQLTableRetrieverQueryEngine(
 sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [21]:
response = query_engine.query("What is current tweet of TSLA stock name? Also, tell the date.")
print(response)

Based on the query results, here's a synthesized response:

"According to recent tweets, Tesla (TSLA) stock has been making headlines. On September 29, 2022, at approximately 11:41 PM GMT, a user shared their experience with a coworker, stating that they expressed their support for Tesla due to its commitment to safety, but were met with disagreement. The tweet read: 'Mainstream media has done an amazing job at brainwashing people. Today at work, we were asked what companies we believe in & I said @Tesla because they make the safest cars & EVERYONE disagreed with me because they heard“they catch on fire & the batteries cost 20k to replace”'. The tweet was posted on September 29, 2022, at 23:41:16 GMT."


In [23]:
response.metadata["result"]


[('Mainstream media has done an amazing job at brainwashing people. Today at work, we were asked what companies we believe in &amp; I said @Tesla because they make the safest cars &amp; EVERYONE disagreed with me because they heard“they catch on fire &amp; the batteries cost 20k to replace”',
  '2022-09-29 23:41:16+00:00')]